# API demo

This is a short demo demonstrating the main functionality of the matscholar API.

### Instantiate the rester

For an API key contact lweston@lbl.gov, jdagdelen@lbl.gov, or amalietrewartha@lbl.gov.

If an API key has already been obtained, the rester is instantiated as follows:

In [ ]:
from matscholar.rest import Rester
rester = Rester(api_key="your-api-key", endpoint="http://api.matscholar.com")

### Resources

##### Searching documents

Our corpus of materials science abstracts can be searched based on text matching (ElasticSearch) or by filtering based on the Named Entities extracted from each document. Entity based searches support the following entity types: material, property, application, descriptor, characterization, synthesis, phase.

To get the raw text of abstracts matching a given query:

In [ ]:
#include text mentioning "light-emitting diode"
example_text = "light-emitting diode"
#include material "GaN" exclude "InN"; include descriptor "thin film"
example_entities = {'material': ['GaN', "-InN"], "descriptor": ["thin film"]}
docs = rester.search_text_with_ents(text=example_text, filters=example_entities, cutoff=10)

In [14]:
docs[0]

{'abstract': 'In this work, a nano-cavity patterned sapphire substrate (nc-PSS) is fabricated by using a self-formed meshed Pt thin film on a c-plane sapphire substrate. The light output power of a GaN-based light emitting diode on the nc-PSS is 45% greater than that of a control light emitting diode that was prepared on a flat c-plane sapphire substrate (f-SS) wafer. The GaN-based light emitting diode that was prepared on the nc-PSS exhibited much less drooping than a GaN-based light-emitting diode that was prepared on a commercial semi-sphere patterned sapphire substrate (r-PSS), mainly because the voids that formed at the cavities at the GaN/nc-PSS interface buffered the stress in the GaN epi-layers that was imposed by the sapphire substrate.',
 'authors': ['Huang, S.W.',
  'Chang, C.C.',
  'Lin, H.Y.',
  'Li, X.F.',
  'Lin, Y.C.',
  'Liu, C.Y.'],
 'doi': '10.1016/j.tsf.2017.03.015',
 'journal': 'Thin Solid Films',
 'link': 'https://www.sciencedirect.com/science/article/pii/S0040609

Note that we can also search on journal names, authors. etc

In [ ]:
#include text mentioning "light-emitting diode"
example_text = "\"Solid State Communications\""
#include material "GaN" exclude "InN"; include descriptor "thin film"
example_entities = {'material': ['GaN', "-InN"], "descriptor": ["thin film"]}
docs = rester.search_text_with_ents(text=example_text, filters=example_entities, cutoff=10)

In [16]:
docs[0]

{'abstract': 'Nitride Zn–Ge thin films were deposited by reaction sputtering. Black conducting (Zn1−xGex)3N2+δ with x≤0.27 was obtained in a compositional range below 29wt% Ge against the total metal content. Greenish pale yellow ZnGeN2 was observed in a range of 30–60wt% Ge. This crystallized in a hexagonal lattice of a comparable size to the isoelectronic GaN. Its solid solution range seems to be very narrow and its band gap was estimated to be about 3.1eV. Ge3N4, like amorphous (Ge1−yZny)3N4−γ with y≤0.43 films, shows pale yellow color above the compositional range of 60wt% Ge.',
 'authors': ['Kikkawa, S', 'Morisaka, H'],
 'doi': '10.1016/S0038-1098(99)00389-0',
 'journal': 'Solid State Communications',
 'link': 'https://www.sciencedirect.com/science/article/pii/S0038109899003890',
 'title': 'RF-sputter deposition of Zn–Ge nitride thin films ',
 'year': '1999'}

##### Searching entities

We have extracted materials-science named entities from nearly 3.5 million materials science absracts.

The extracted named entities for each document associated with a query are returned by the search_ents method. This method takes as input a dictionary with entity types as keys and a list of entities as values. For example, to find all of the entities that co-occur with the query below:

In [ ]:
#get entities for documnets mentioning material "BaZrO3" and descriptor "single crystal"
docs = rester.search_ents(query={"material": ["BaZrO3"], "descriptor": ["single crystal"]})
docs[0]

This wil return a list of dictionaries representing documents matching the query; each dict will contain the DOI as well as each unique entity found in the corresponding abstract.

A summary of the entities associated with a query can be generated using the get_summary method. To get statistics for entities co-occuring with GaN:

In [ ]:
summary = rester.get_summary(query={"material": ["GaN"]})

This will return a dictionary with entity types as keys; the values will be a list of the top entities that occur in documents matching the query, each item in the list will be [entity, document count, fraction].

To show the synthesis methods from the summary:

In [ ]:
summary["SMT"][:5]

To perform a fast literature review, the materials_search_ents method may be used. For a chosen application, this will return a list of all materials that co-occur with that application in our corpus. For example, to see which materials co-occur with the word thermoelectric in a document,

In [ ]:
mat_list = rester.materials_search_ents(["thermoelectric"], elements=["-Pb"], cutoff=None)
for mat, counts, dois in mat_list[:20]:
    print(mat, counts, dois[:3])

The above search will find all materials co-occurring with thermoelectric that do not contain lead. The result will be a list, with each element containing a list of [material, co-occurence counts, co-occurrence dois].

##### Word embeddings

Materials science word embeddings trained using word2vec.

To get the word embedding for a given word:

In [ ]:
embedding = rester.get_embedding("photovoltaics")
embedding.keys()

This will return a dict containing the embedding. The word embedding will be a 200-dimensional array.

The rester also has a close_words method (based on cosine similarity of embeddings) which can be used to explore the semantic similarity of materials science terms; this approach can be used discover materials for a new application (as outlined in the reference above),

To find words with a similar embedding to photovolatic:

In [ ]:
close_words = rester.close_words("photovoltaics", top_k=1000)
close_words["close_words"][:10]

This will return the 1000 closest words to photovoltaics. The result will be a dictionary containing the close words and their cosine similarity to the input word.

##### Named entity recognition

In addition to the pre-processed entities present in our corpus, users can performed Named Entity Recognition on any raw materials science text.

The input should be a list of documents with the text represented as a string:

In [ ]:
doc_1 = "The bands gap of TiO2 is 3.2 eV. This was measured via photoluminescence."
doc_2 = "We deposit GaN thin films using MOCVD"
docs = [doc_1, doc_2] 
tagged_docs = rester.get_ner_tags(docs, return_type="concatenated")
tagged_docs[0]

The argument return_type may be set to iob, concatenated, or normalized. The latter will replace entities with their most frequently occurring synonym. A list of tagged documents will be returned. Each doc is a list of sentences; each sentence is a list of (word, tag) pairs.

### References

If you use the matscholar API in your research, please cite the following papers:

[1] V. Tshitoyan et al., Nature 571, 95 (2019). 

[2] L. Weston et al., Submitted to J. Chem. Inf. Model., https://doi.org/10.26434/chemrxiv.8226068.v1